In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import numpy as np
from gensim.models import word2vec
from gensim.corpora.dictionary import Dictionary
import pickle
from keras.preprocessing import sequence

import time, datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import gc
from sklearn.model_selection import GridSearchCV
import jieba, pdb
from sklearn.decomposition import PCA
from scipy import stats

jieba.set_dictionary('jieba_dict/dict.txt.big')
# load stopwords set
stopword_set = set()
with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

model = word2vec.Word2Vec.load("word2vec2.model")


def create_dictionaries(p_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(p_model.wv.vocab.keys(), allow_update=True)
    w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
    return w2indx, w2vec

def Convert_uniqueid(x, sets):
    return np.where(sets==x)[0][0]

def parsePathalias(x):
    if x == 'None':
        return 0
    else:
        return len(x)

def Convert_Date(x):
    timestamp = x
    timeArray = time.localtime(timestamp)
    datetime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    return datetime

def Date2Ticks(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = str(Year+'/'+Month+'/'+Day)
    return time.mktime(datetime.datetime.strptime(date1, "%Y/%m/%d").timetuple())

index_dict, word_vectors= create_dictionaries(model)
output = open("wordwmbedding.pkl", 'wb')
pickle.dump(index_dict, output)  # 索引字典
pickle.dump(word_vectors, output)  # 词向量字典
output.close()



C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [2]:
# load dataset
df_tag = pd.read_csv("train/train_tags2.txt", delimiter=",",encoding = "utf-8")
df_add = pd.read_csv("train/train_additional.txt",delimiter=" ")
df_cat = pd.read_csv("train/train_category.txt",delimiter=" ")
df_ts = pd.read_csv("train/train_temporalspatial.txt",delimiter=" ")
df_label = pd.read_csv("train/train_label.txt",delimiter=" ")

'''
Cat: Uid Pid Category Subcategory Concept
TAG: Uid Pid Title Mediatype Alltags
TS:  Uid Pid Postdate Latitude Longitude Geoaccuracy
Add: Uid Pid Pathalias Ispublic Mediastatus

'''
# date Conversion
uidset =      np.unique(np.asarray(df_cat['Uid'].values.tolist()))
pidset =      np.unique(np.asarray(df_cat['Pid'].values.tolist()))
Category =    np.unique(np.asarray(df_cat['Category'].values.tolist()))
Subcategory = np.unique(np.asarray(df_cat['Subcategory'].values.tolist()))
Concept =     np.unique(np.asarray(df_cat['Concept'].values.tolist()))


month = {'Jan': '01', 'Feb': '02' , 'Mar':'03' ,'Apr': '04', 
'May': '05', 'Jun': '06' , 'Jul': '07' , 'Aug':'08', 
'Sep':'09', 'Oct':'10' , 'Nov':'11', 'Dec':'12' }

# group data
df_cat['Uid']=df_cat.Uid.apply(lambda x: Convert_uniqueid(x, uidset))
df_cat['Pid']=df_cat.Pid.apply(lambda x: Convert_uniqueid(x, pidset))
df_cat['Category']= df_cat.Category.apply(lambda x: Convert_uniqueid(x, Category))
df_cat['Subcategory']= df_cat.Subcategory.apply(lambda x:  Convert_uniqueid(x, Subcategory))
df_cat['Concept']= df_cat.Concept.apply(lambda x: Convert_uniqueid(x, Concept))

df_add['Uid']=df_add.Uid.apply(lambda x: Convert_uniqueid(x, uidset))
df_add['Pid']=df_add.Pid.apply(lambda x: Convert_uniqueid(x, pidset))
df_add['Pathalias']=df_add.Pathalias.apply(lambda x: parsePathalias(x))
df_add['Ispublic']=df_add.Ispublic.apply(lambda x: int(x))
df_add['Mediastatus']=df_add.Mediastatus.apply(lambda x: len(x))


df_ts['Uid']=df_ts.Uid.apply(lambda x: Convert_uniqueid(x, uidset))
df_ts['Pid']=df_ts.Pid.apply(lambda x: Convert_uniqueid(x, pidset))
df_ts['Postdate']=df_ts.Pid.apply(lambda x: int(x))
df_ts['Latitude']=df_ts.Pid.apply(lambda x: float(x))
df_ts['Longitude']=df_ts.Pid.apply(lambda x: float(x))
df_ts['Geoaccuracy']=df_ts.Pid.apply(lambda x: float(x))

df_tag['Uid']=df_tag.Uid.apply(lambda x: Convert_uniqueid(x, uidset))
df_tag['Pid']=df_tag.Pid.apply(lambda x: Convert_uniqueid(x, pidset))
df_tag['Title']= df_tag.Title.apply(lambda x: len(x))

df_label['score'] = df_label.score.apply(lambda x: float(x))

# df_airline = df_airline.drop_duplicates(subset='group_id', keep='first', inplace=False)

# group_used_cols=['group_id','Begin_Date','Begin_Tick','days','Area','SubLine','price', 'product_name']
# df_group_0 = df_group[group_used_cols].merge(df_airline, on='group_id')

# # for order data
# df_order_1['Order_Date']=df_order_1.order_date.apply(lambda x: Convert_Date(x))
# df_order_1['Order_Tick']=df_order_1.order_date.apply(lambda x: Date2Ticks(x))
# df_order_1['order_id']=df_order_1.order_id.apply(lambda x: Convert_orderid(x))
# df_order_1['Source_1']= df_order_1.source_1.apply(lambda x: int(x[11:]))
# df_order_1['Source_2']= df_order_1.source_2.apply(lambda x: int(x[11:]))
# df_order_1['Unit']= df_order_1.unit.apply(lambda x: int(x[11:]))
# df_order_1['Begin_Date']=pd.to_datetime(df_order_1['Begin_Date'])
# df_order_1['Order_Date']=pd.to_datetime(df_order_1['Order_Date'])
# df_order_1['PreDays']=(df_order_1['Begin_Date']-df_order_1['Order_Date']).dt.days
# df_order_1['Begin_Date_Weekday']= df_order_1['Begin_Date'].dt.dayofweek
# df_order_1['Order_Date_Weekday']= df_order_1['Order_Date'].dt.dayofweek
# df_order_1['Return_Date_Weekday']= (df_order_1['Begin_Date'].dt.dayofweek+df_order_1['days'])%7
# df_order_1['tick_diff'] = (df_order_1['Begin_Tick'] - df_order_1['Order_Tick'])/10000
# df_order_1['price'] = df_order_1['price']/1000

# order_used_columns=['order_id', 'group_id','tick_diff', 'Source_1', 'Source_2', 'Unit',
# 'people_amount', 'Begin_Tick','days', 'Order_Tick', 'Area', 'SubLine', 'price','PreDays','Begin_Date_Weekday', 
# 'Order_Date_Weekday', 'Return_Date_Weekday', 'fly_t', 'fly_date',
# "src_airport", "arrive_t", "arrive_date", "dst_airport", 'product_name']

# df_order_2=df_order_1[order_used_columns].merge(df_day_schedule[['group_id','title']], on='group_id')


In [15]:
print("Got %d training with %d labels" % (len(df_cat), len(df_label)))


df_all = df_cat.merge(df_tag, on=['Uid', 'Pid'])
df_all2 = df_ts.merge(df_all, on=['Uid', 'Pid'])
df_all = df_tag.merge(df_all2, on=['Uid', 'Pid'])

# train/test data
print("Got %d training with %d labels" % (len(df_all), len(df_label)))

# del df_all['Pid'] 
# del df_all['Uid']

Got 305613 training with 305613 labels
Got 305613 training with 305613 labels


In [4]:
# train/test data
# df_train_1=df_train.merge(df_order_2,on='order_id')
# df_result_1=df_result.merge(df_order_2,on='order_id')
# df_train_1.to_csv('myTrain2.csv', index=False)
# df_result_1.to_csv('myTest2.csv', index=False)

# Y=df_train_1['deal_or_not'].values.tolist()
# swX_tmp = (df_train_1['product_name']).values.tolist()
# Xid = df_train_1['order_id'].values.tolist()
# del df_train_1['deal_or_not'] 
# del df_train_1['product_name']
# del df_train_1['group_id'] 
# del df_train_1['order_id']
# X = df_train_1.values.tolist()

# rid = df_result_1['order_id'].values.tolist()
# swrx = (df_result_1['product_name']).values.tolist()
# del df_result_1['product_name']
# del df_result_1['deal_or_not']
# del df_result_1['order_id']
# del df_result_1['group_id']

# rx = df_result_1.values.tolist()


# sX, sY, Xid =np.asarray(X), np.asarray(Y), np.asarray(Xid)
# rx,rid = np.asarray(rx), np.asarray(rid)
# X,Y, swX=[],[], []
# for i in range(len(sY)):
#    # if (int(Xid[i])<=204000):
#         X.append(sX[i,:])
#         Y.append(sY[i])
#         swX.append(swX_tmp[i])
# X, Y = np.array(X), np.array(Y)


# def text_to_index_array(p_new_dic, p_sen):  # 文本转为索引数字模式
#     new_sentences = []
#     for sen in p_sen:
#         new_sen = []
#         for word in str(sen):
#             try:
#                 new_sen.append(p_new_dic[word])  # 单词转索引数字
#             except:
#                 new_sen.append(0)  # 索引字典里没有的词转为数字0
#         new_sentences.append(new_sen)

#     return np.array(new_sentences)


# wX = text_to_index_array(index_dict, swX)
# wrx = text_to_index_array(index_dict, swrx)
# wX = sequence.pad_sequences(wX, maxlen=60)
# wrx = sequence.pad_sequences(wrx, maxlen=60)

# from sklearn.preprocessing import StandardScaler, PowerTransformer

# pt_X = PowerTransformer(method='yeo-johnson', standardize=False)
# sc_y = StandardScaler()
# sc_X = StandardScaler()
# X = sc_X.fit_transform(X)
# rx = sc_X.transform(rx)

# # X=np.concatenate([X, wX], axis=1)
# # rx=np.concatenate([rx, wrx], axis=1)
# # xlen=len(X)
# # from sklearn.preprocessing import normalize
# # Xtmp=normalize(np.concatenate([X, rx], axis=0),norm='max', axis=0)
# # X=Xtmp[:xlen]
# # rx=Xtmp[xlen:]

# print(X.shape)

# # np.save("data.npy", [X,Y,rx])
# # [X,Y,rx] = np.load("data.npy")

In [8]:
Y.shape

(305613,)

In [16]:
folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=666)

X = df_all.values
Y = df_label.values
Y = np.reshape(Y,[-1])
Y2=np.asarray(Y,np.int)

oof_preds = np.zeros(X.shape[0])
# sub_preds = np.zeros(rx.shape[0])
feature_importance_df = pd.DataFrame()


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, Y2)):

    train_x, train_y = X[train_idx,:], Y[train_idx]
    valid_x, valid_y = X[valid_idx,:], Y[valid_idx]

    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
    'nthread': 32, 'boosting_type': 'dart','objective': 'regression', 'metric': ['mae','mse',stats.spearmanr], 
        'learning_rate': 0.01, 'num_leaves': 70,
    'max_depth': 9, 'subsample': 0.8, 'feature_fraction': 0.9, 
        'min_split_gain': 0.09, 'min_child_weight': 9.5,
#         'device_type':'gpu',
    # parameters for dart
    'drop_rate':0.5, 'skip_drop':0.5, 'max_drop':6, 'uniform_drop':False, 
        'xgboost_dart_mode':True, 'drop_seed':1 }
    

    if n_fold >= 0:
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=15000, valid_sets=[dval], early_stopping_rounds=2500, verbose_eval=500)


        tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
        oof_preds[valid_idx] = tmp_valid
      
        # Make the feature importance dataframe
        gain = bst.feature_importance('gain')
        fold_importance_df = pd.DataFrame({'feature':bst.feature_name(),'split':bst.feature_importance('split'),
                                           'gain':100*gain/gain.sum(),  'fold':n_fold,}).sort_values('gain',ascending=False)

        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d rank corr : %.6f' % (n_fold + 1, stats.spearmanr(valid_y, oof_preds[valid_idx])[0]))

        del bst, train_x, train_y, valid_x, valid_y
        


# app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
# predF=pd.DataFrame({"order_id":rid, "deal_or_not":sub_preds})
# preds = pd.DataFrame({"order_id":app_test['order_id']})
# preds=preds.merge(predF, on="order_id",how="outer")
# preds['deal_or_not'] = preds.deal_or_not.apply(lambda x: nanMapping(x))
# preds.to_csv("output/lgb_dart_" + str(roc_auc_score(Y, oof_preds)) + ".csv", index=False)

Train Index: [     0      2      3 ... 305610 305611 305612] ,Val Index: [     1     17     48 ... 305582 305596 305605]
Training until validation scores don't improve for 2500 rounds.
[500]	valid_0's l1: 1.47614	valid_0's l2: 3.67833
[1000]	valid_0's l1: 1.36509	valid_0's l2: 3.26465
[1500]	valid_0's l1: 1.30175	valid_0's l2: 3.04285
[2000]	valid_0's l1: 1.25349	valid_0's l2: 2.88294
[2500]	valid_0's l1: 1.21357	valid_0's l2: 2.7559
[3000]	valid_0's l1: 1.17963	valid_0's l2: 2.65168
[3500]	valid_0's l1: 1.15112	valid_0's l2: 2.5642
[4000]	valid_0's l1: 1.126	valid_0's l2: 2.49115
[4500]	valid_0's l1: 1.10501	valid_0's l2: 2.42967
[5000]	valid_0's l1: 1.08777	valid_0's l2: 2.37902
[5500]	valid_0's l1: 1.07094	valid_0's l2: 2.32978
[6000]	valid_0's l1: 1.05681	valid_0's l2: 2.29054
[6500]	valid_0's l1: 1.04187	valid_0's l2: 2.24929
[7000]	valid_0's l1: 1.02833	valid_0's l2: 2.21337
[7500]	valid_0's l1: 1.01709	valid_0's l2: 2.18386
[8000]	valid_0's l1: 1.00674	valid_0's l2: 2.15649
[850

[6500]	valid_0's l1: 1.04606	valid_0's l2: 2.23756
[7000]	valid_0's l1: 1.03305	valid_0's l2: 2.20194
[7500]	valid_0's l1: 1.02136	valid_0's l2: 2.16982
[8000]	valid_0's l1: 1.0103	valid_0's l2: 2.13976
[8500]	valid_0's l1: 0.999756	valid_0's l2: 2.11201
[9000]	valid_0's l1: 0.989963	valid_0's l2: 2.08658
[9500]	valid_0's l1: 0.980943	valid_0's l2: 2.06354
[10000]	valid_0's l1: 0.971904	valid_0's l2: 2.03998
[10500]	valid_0's l1: 0.96284	valid_0's l2: 2.01564
[11000]	valid_0's l1: 0.955855	valid_0's l2: 1.99764
[11500]	valid_0's l1: 0.949387	valid_0's l2: 1.98168
[12000]	valid_0's l1: 0.943532	valid_0's l2: 1.96773
[12500]	valid_0's l1: 0.937805	valid_0's l2: 1.95421
[13000]	valid_0's l1: 0.931209	valid_0's l2: 1.93822
[13500]	valid_0's l1: 0.925022	valid_0's l2: 1.923
[14000]	valid_0's l1: 0.919974	valid_0's l2: 1.91069
[14500]	valid_0's l1: 0.9153	valid_0's l2: 1.89985
[15000]	valid_0's l1: 0.910934	valid_0's l2: 1.88939
Did not meet early stopping. Best iteration is:
[15000]	valid_0

[14000]	valid_0's l1: 0.918821	valid_0's l2: 1.91481
[14500]	valid_0's l1: 0.913568	valid_0's l2: 1.90226
[15000]	valid_0's l1: 0.908873	valid_0's l2: 1.89066
Did not meet early stopping. Best iteration is:
[15000]	valid_0's l1: 0.908873	valid_0's l2: 1.89066
Fold  9 rank corr : 0.838454
Train Index: [     0      1      2 ... 305610 305611 305612] ,Val Index: [     3      9     10 ... 305591 305595 305598]
Training until validation scores don't improve for 2500 rounds.
[500]	valid_0's l1: 1.48093	valid_0's l2: 3.6523
[1000]	valid_0's l1: 1.37155	valid_0's l2: 3.24436
[1500]	valid_0's l1: 1.30806	valid_0's l2: 3.02152
[2000]	valid_0's l1: 1.25691	valid_0's l2: 2.85283
[2500]	valid_0's l1: 1.21789	valid_0's l2: 2.72925
[3000]	valid_0's l1: 1.1838	valid_0's l2: 2.62806
[3500]	valid_0's l1: 1.15712	valid_0's l2: 2.54899
[4000]	valid_0's l1: 1.13032	valid_0's l2: 2.4699
[4500]	valid_0's l1: 1.10876	valid_0's l2: 2.40649
[5000]	valid_0's l1: 1.09038	valid_0's l2: 2.35348
[5500]	valid_0's l1: